# Metadata

**L1 Taxonomy** - Security

**L2 Taxonomy** - Authorization

**Subtopic** - Implementing resource ownership checks (user can only modify their own resources) at the service layer

**Use Case** - Develop a Python module that simulates a service layer for a simple file-based resource system. The module should include functions for creating, reading, updating, and deleting resources, represented as files within a local directory. Each resource should be owned by a user, and the module should enforce that a user can only modify their own resources. The system should be able to handle multiple users, each with their own set of resources.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
```


# Prompt


### Prompt

**Problem Description**
Implement a module named `ResourceService` that keeps every user’s text resources inside a single JSON file: `<root>/<user_id>.json`.
Each resource holds a list of immutable revisions, and every write appends a new revision.
The module must

* guarantee that only the owner can change or delete a resource,
* maintain a tamper‑evident audit log (`audit.log`) with a SHA‑256 hash chain,
* stay correct when ten‑thousand concurrent threads perform mixed operations,
* reject path‑traversal attacks or symlink tricks that attempt to escape `<root>`.

**Input Format and Constraints**

* `user_id` – non‑empty string, no path separators, case‑sensitive.
* `resource_name` – non‑empty UTF‑8, ends with “.txt”, no path separators, normalised with NFC, ≤ 255 bytes after normalisation, unique for a user when compared case‑insensitively after normalisation.
* `revision_id` – a hex string returned by this service; callers must not invent one.
* `content` – arbitrary UTF‑8 text.
  Validation is required; wrong inputs raise the exceptions listed under Edge‑case Behaviour.

**Expected Output Format**
All public methods return Python primitives (str, list, dict, or None) or raise a documented exception.
They must not print or log to stdout/stderr.

**Illustrative Example**

```
svc = ResourceService("./data")

rev1 = svc.create_resource("alice", "note.txt", "first")
rev2 = svc.update_resource("alice", "note.txt", "second")

latest = svc.read_resource("alice", "note.txt")          # "second"
older  = svc.read_resource("alice", "note.txt", rev1)    # "first"

history = svc.list_revisions("alice", "note.txt")        # [rev1, rev2]
ok = svc.verify_audit_log()                              # True
```



# Requirements



### Requirements

**Explicit and Implicit Points**

* Each user file is a JSON object mapping normalised slugs to a list of revision records: `{"slug": [{"id": "...", "content": "..."} , ...]}`.
* `create_resource` fails if any revision exists for that slug.
* Each update appends a new revision and returns its `revision_id`.
* All writes go through a temp file followed by `os.replace` for crash‑safety.
* Use a per‑user lock file (e.g. `<user_id>.lock`) to give writers exclusive access; readers spin until the lock disappears (timeout 200 ms).
* Append every successful public operation to `audit.log` as one JSON line containing a SHA‑256 hash of the previous line plus the current event.
* The service must detect any break in the hash chain: `verify_audit_log()` returns `False` if tampering is found.
* Resolve real paths after each file action; abort if the resolved path leaves `<root>`.

**Solution Expectations**

* Standard library only, Python 3.10+.
* Pass a stress test: 500 threads × 20 adversarial operations within two seconds wall time on a 32‑core machine.
* Deterministic behaviour; no reliance on `time.sleep`–based randomness.

**Signatures of Expected Functions**

```python
class ResourceService:
    def __init__(self, root_dir: str) -> None: ...
    def create_resource(self, user_id: str, resource_name: str, content: str) -> str: ...
    def read_resource(
        self,
        user_id: str,
        resource_name: str,
        revision_id: str | None = None
    ) -> str: ...
    def update_resource(
        self,
        user_id: str,
        resource_name: str,
        new_content: str
    ) -> str: ...
    def delete_resource(self, user_id: str, resource_name: str) -> None: ...
    def list_resources(self, user_id: str) -> list[str]: ...
    def list_revisions(self, user_id: str, resource_name: str) -> list[str]: ...
    def verify_audit_log(self) -> bool: ...
```

**Edge‑case Behaviour**
• Bad `user_id` or `resource_name` – raise `ValueError`.
• Duplicate resource creation – raise `FileExistsError`.
• Missing resource on read, update, delete, or list\_revisions – raise `FileNotFoundError`.
• Attempt to access a path outside `<root>` or via symlink – raise `PermissionError`.
• Lock acquisition exceeds 200 ms – raise `TimeoutError`.
• `verify_audit_log()` returns `False` (no exception) if the hash chain is broken.

**Constraints**

* No third‑party libraries.
* Never call `subprocess`, `os.system`, or similar.
* Aim for roughly constant‑time filesystem work per public call (ignoring unavoidable JSON parsing).
* All cryptographic hashes must use `hashlib.sha256`.

**Important Notes**
All constraints are enforced during testing. The harness feeds invalid inputs, Unicode collision attempts, concurrent symlink races, and deliberate audit‑log edits. Your code must raise exactly the exceptions listed above or return the specified values.




In [ ]:
# code

import os
import json
import time
import secrets
import hashlib
import unicodedata
from contextlib import contextmanager
from typing import Optional, List, Dict, Any


class ResourceService:
    def __init__(self, root_dir: str) -> None:
        self.root = os.path.abspath(root_dir)
        os.makedirs(self.root, exist_ok=True)
        self.audit_path = os.path.join(self.root, "audit.log")
        self.audit_lock = os.path.join(self.root, "audit.lock")

    # ------------------------------------------------------------------ public

    def create_resource(
        self, user_id: str, resource_name: str, content: str
    ) -> str:
        uid = self._check_user_id(user_id)
        name = self._check_resource_name(resource_name)
        slug = name.casefold()

        with self._user_lock(uid):
            data = self._read_user_json(uid)
            if slug in data:
                raise FileExistsError(
                    f"{resource_name!r} exists for user {uid!r}"
                )
            rev = secrets.token_hex(16)
            data[slug] = {"name": name, "revs": [{"id": rev, "text": content}]}
            self._write_user_json(uid, data)
        # ── critical section ended ──
        self._log_event(
            {"action": "create", "user": uid, "resource": name, "rev": rev}
        )
        return rev

    def read_resource(
        self,
        user_id: str,
        resource_name: str,
        revision_id: Optional[str] = None
    ) -> str:
        uid = self._check_user_id(user_id)
        name = self._check_resource_name(resource_name)
        slug = name.casefold()

        self._wait_unlock(uid)
        data = self._read_user_json(uid)
        if slug not in data:
            raise FileNotFoundError(
                f"{resource_name!r} not found for user {uid!r}"
            )
        revs = data[slug]["revs"]
        if revision_id is None:
            return revs[-1]["text"]
        for r in revs:
            if r["id"] == revision_id:
                return r["text"]
        raise FileNotFoundError(
            f"Revision {revision_id!r} not found for {resource_name!r}"
        )

    def update_resource(
        self, user_id: str, resource_name: str, new_content: str
    ) -> str:
        uid = self._check_user_id(user_id)
        name = self._check_resource_name(resource_name)
        slug = name.casefold()

        with self._user_lock(uid):
            data = self._read_user_json(uid)
            if slug not in data:
                raise FileNotFoundError(
                    f"{resource_name!r} not found for user {uid!r}"
                )
            rev = secrets.token_hex(16)
            data[slug]["revs"].append({"id": rev, "text": new_content})
            self._write_user_json(uid, data)
        # ── critical section ended ──
        self._log_event(
            {"action": "update", "user": uid, "resource": name, "rev": rev}
        )
        return rev

    def delete_resource(self, user_id: str, resource_name: str) -> None:
        uid = self._check_user_id(user_id)
        name = self._check_resource_name(resource_name)
        slug = name.casefold()

        with self._user_lock(uid):
            data = self._read_user_json(uid)
            if slug not in data:
                raise FileNotFoundError(
                    f"{resource_name!r} not found for user {uid!r}"
                )
            del data[slug]
            self._write_user_json(uid, data)
        # ── critical section ended ──
        self._log_event(
            {"action": "delete", "user": uid, "resource": name}
        )

    def list_resources(self, user_id: str) -> List[str]:
        uid = self._check_user_id(user_id)
        self._wait_unlock(uid)
        data = self._read_user_json(uid)
        return [info["name"] for info in data.values()]

    def list_revisions(
        self, user_id: str, resource_name: str
    ) -> List[str]:
        uid = self._check_user_id(user_id)
        name = self._check_resource_name(resource_name)
        slug = name.casefold()

        self._wait_unlock(uid)
        data = self._read_user_json(uid)
        if slug not in data:
            raise FileNotFoundError(
                f"{resource_name!r} not found for user {uid!r}"
            )
        return [r["id"] for r in data[slug]["revs"]]

    def verify_audit_log(self) -> bool:
        if not os.path.exists(self.audit_path):
            return True
        prev = ""
        try:
            with open(self.audit_path, "r", encoding="utf-8") as f:
                for raw in f:
                    entry = json.loads(raw)
                    expected = hashlib.sha256(
                        (prev + json.dumps(entry["event"],
                                           sort_keys=True)).encode()
                    ).hexdigest()
                    if entry.get("hash") != expected:
                        return False
                    prev = raw.rstrip("\n")
        except Exception:
            return False
        return True

    # ----------------------------------------------------------------- helpers

    def _check_user_id(self, uid: str) -> str:
        if not uid or os.sep in uid:
            raise ValueError("Invalid user_id")
        path = os.path.realpath(os.path.join(self.root, f"{uid}.json"))
        if not path.startswith(self.root + os.sep):
            raise PermissionError("Path escape")
        return uid

    def _check_resource_name(self, name: str) -> str:
        # accept any case “.txt”, forbid path separators
        if (
            not name
            or "/" in name
            or "\\" in name
            or not name.lower().endswith(".txt")
        ):
            raise ValueError("Invalid resource_name")
        norm = unicodedata.normalize("NFC", name)
        if len(norm.encode("utf-8")) > 255:
            raise ValueError("Name too long")
        return norm

    def _read_user_json(self, uid: str) -> Dict[str, Any]:
        path = os.path.join(self.root, f"{uid}.json")
        if not os.path.exists(path):
            return {}
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)

    def _write_user_json(self, uid: str, data: Dict[str, Any]) -> None:
        path = os.path.join(self.root, f"{uid}.json")
        tmp = path + ".tmp"
        with open(tmp, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)
        os.replace(tmp, path)

    def _wait_unlock(self, uid: str, timeout: float = 0.2) -> None:
        lock = os.path.join(self.root, f"{uid}.lock")
        start = time.monotonic()
        while os.path.exists(lock):
            if time.monotonic() - start > timeout:
                raise TimeoutError("Lock wait timed out")
            time.sleep(0.001)

    @contextmanager
    def _user_lock(self, uid: str, timeout: float = 0.2):
        lock = os.path.join(self.root, f"{uid}.lock")
        start = time.monotonic()
        while True:
            try:
                fd = os.open(lock, os.O_CREAT | os.O_EXCL | os.O_WRONLY)
                os.close(fd)
                break
            except FileExistsError:
                if time.monotonic() - start > timeout:
                    raise TimeoutError("Could not acquire user lock")
                time.sleep(0.001)
        try:
            yield
        finally:
            try:
                os.remove(lock)
            except OSError:
                pass

    def _log_event(self, event: dict) -> None:
        start = time.monotonic()
        while True:
            try:
                fd = os.open(
                    self.audit_lock, os.O_CREAT | os.O_EXCL | os.O_WRONLY
                )
                os.close(fd)
                break
            except FileExistsError:
                if time.monotonic() - start > 0.2:
                    raise TimeoutError("Audit lock timeout")
                time.sleep(0.001)

        prev_line = ""
        if os.path.exists(self.audit_path):
            with open(self.audit_path, "r", encoding="utf-8") as f:
                lines = f.read().splitlines()
                if lines:
                    prev_line = lines[-1]

        payload = json.dumps(event, sort_keys=True)
        h = hashlib.sha256((prev_line + payload).encode()).hexdigest()
        record = {"hash": h, "event": event}
        line = json.dumps(record, ensure_ascii=False)

        with open(self.audit_path, "a", encoding="utf-8") as f:
            f.write(line + "\n")

        try:
            os.remove(self.audit_lock)
        except OSError:
            pass


In [ ]:
# tests

import os
import json
import time
import threading
import tempfile
import unittest
from pathlib import Path

# The implementation to be tested must live in main.py
from main import ResourceService


# ---------- helpers ---------- #

def new_service():
    """Return a fresh ResourceService rooted in a brand‑new temp dir
    and the pathlib.Path object for that directory."""
    tmpdir = tempfile.TemporaryDirectory()
    root = Path(tmpdir.name)
    return ResourceService(str(root)), root, tmpdir


def read_audit(root: Path) -> list[dict]:
    """Return audit.log lines already parsed as JSON."""
    log_file = root / "audit.log"
    if not log_file.exists():
        return []
    return [json.loads(l) for l in log_file.read_text().splitlines()]


# ---------- test‑cases ---------- #

class ResourceServiceBasics(unittest.TestCase):
    def setUp(self) -> None:
        self.svc, self.root, self._tmp = new_service()

    def tearDown(self) -> None:
        # Ensure tempdir is cleaned even if the test fails.
        self._tmp.cleanup()

    # ----- normal workflow ----- #
    def test_create_read_update_delete_cycle(self):
        rid1 = self.svc.create_resource("alice", "note.txt", "first")
        self.assertIsInstance(rid1, str)
        self.assertEqual(self.svc.read_resource("alice", "note.txt"), "first")

        rid2 = self.svc.update_resource("alice", "note.txt", "second")
        self.assertNotEqual(rid1, rid2)
        self.assertEqual(self.svc.read_resource("alice", "note.txt"), "second")
        self.assertEqual(self.svc.read_resource("alice", "note.txt", rid1), "first")

        revs = self.svc.list_revisions("alice", "note.txt")
        self.assertEqual(revs, [rid1, rid2])

        resources = self.svc.list_resources("alice")
        self.assertEqual(resources, ["note.txt"])

        # audit log must verify
        self.assertTrue(self.svc.verify_audit_log())

        # delete and ensure it is gone
        self.svc.delete_resource("alice", "note.txt")
        with self.assertRaises(FileNotFoundError):
            self.svc.read_resource("alice", "note.txt")

    # ----- duplicate and case‑insensitive slug clash ----- #
    def test_duplicate_resource_raises(self):
        self.svc.create_resource("alice", "Hello.txt", "a")
        with self.assertRaises(FileExistsError):
            self.svc.create_resource("alice", "hELLo.TXT", "b")

    # ----- edge‑case: bad inputs ----- #
    def test_invalid_user_and_resource_names(self):
        with self.assertRaises(ValueError):
            self.svc.create_resource("al/ice", "note.txt", "x")          # slash inside user
        with self.assertRaises(ValueError):
            self.svc.create_resource("bob", "note", "x")                 # missing .txt
        with self.assertRaises(ValueError):
            self.svc.create_resource("bob", "../note.txt", "x")          # traversal chars
        long_name = "a" * 260 + ".txt"
        with self.assertRaises(ValueError):
            self.svc.create_resource("bob", long_name, "x")

    # ----- missing resource operations ----- #
    def test_missing_resource_errors(self):
        with self.assertRaises(FileNotFoundError):
            self.svc.read_resource("nobody", "ghost.txt")
        with self.assertRaises(FileNotFoundError):
            self.svc.update_resource("nobody", "ghost.txt", "boo")
        with self.assertRaises(FileNotFoundError):
            self.svc.delete_resource("nobody", "ghost.txt")
        with self.assertRaises(FileNotFoundError):
            self.svc.list_revisions("nobody", "ghost.txt")

    # ----- path‑traversal / symlink attack ----- #
    def test_symlink_escape_prevention(self):
        # Forge a symlink inside <root> that points outside
        outside = Path(tempfile.gettempdir()) / "evil.json"
        outside.write_text("{}")
        (self.root / "bob.json").symlink_to(outside)
        with self.assertRaises(PermissionError):
            self.svc.create_resource("bob", "note.txt", "should fail")

    # ----- audit‑log tamper detection ----- #
    def test_audit_log_tampering_detected(self):
        self.svc.create_resource("alice", "note.txt", "hi")
        self.assertTrue(self.svc.verify_audit_log())      # baseline OK

        # Break the hash chain: flip one bit in the 2nd char of 2nd line
        log_file = self.root / "audit.log"
        lines = log_file.read_bytes().splitlines()
        if len(lines) < 2:   # paranoia guard
            self.skipTest("need at least two lines to tamper")
        tampered = bytearray(lines[1])
        tampered[2] ^= 0x01
        lines[1] = bytes(tampered)
        log_file.write_bytes(b"\n".join(lines))

        self.assertFalse(self.svc.verify_audit_log())

    # ----- concurrent access with per‑user locking ----- #
    def test_concurrent_updates_consistent(self):
        rid0 = self.svc.create_resource("alice", "note.txt", "v0")

        # 20 threads each append one revision
        def worker(i):
            self.svc.update_resource("alice", "note.txt", f"v{i}")

        threads = [threading.Thread(target=worker, args=(i,)) for i in range(20)]
        for t in threads:
            t.start()
        for t in threads:
            t.join()

        revs = self.svc.list_revisions("alice", "note.txt")
        # initial + 20 updates
        self.assertEqual(len(revs), 1 + 20)
        self.assertEqual(self.svc.read_resource("alice", "note.txt"), "v19")
        self.assertTrue(self.svc.verify_audit_log())


# Run the tests when executed directly
if __name__ == "__main__":
    unittest.main(verbosity=2)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/97dbf10b-6433-4ec6-a524-1e9f0c692a26?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F80889%2Fview

```python
def create_resource(self, user_id: str, resource_name: str, content: str) -> str:
    user_id = self._validate_user_id(user_id)
    normalized_name = self._validate_resource_name(resource_name)
    slug = self._slugify(normalized_name)

    # Acquire writer lock
    with self._acquire_user_lock(user_id):
        user_data = self._load_user_data(user_id)

        if slug in user_data:
            # Resource already exists
            raise FileExistsError(f"Resource {resource_name!r} already exists for user {user_id!r}.")

        # Create the new resource’s revision list
        rev_id = self._generate_revision_id()
        user_data[slug] = {
            "name": normalized_name,  # store the canonical name
            "revisions": [
                {
                    "id": rev_id,
                    "content": content
                }
            ]
        }

        self._write_user_data(user_id, user_data)
        # Append operation to audit log
        self._append_audit_log({
            "op": "create_resource",
            "user_id": user_id,
            "resource_name": resource_name,
            "revision_id": rev_id
        })
        return rev_id

def read_resource(
    self,
    user_id: str,
    resource_name: str,
    revision_id: Optional[str] = None
) -> str:
    user_id = self._validate_user_id(user_id)
    normalized_name = self._validate_resource_name(resource_name)
    slug = self._slugify(normalized_name)

    # Readers spin until lock disappears (or 200ms passes)
    self._wait_for_lock_to_disappear(user_id)

    user_data = self._load_user_data(user_id)
    if slug not in user_data:
        raise FileNotFoundError(f"Resource {resource_name!r} not found for user {user_id!r}.")

    rev_list = user_data[slug]["revisions"]
    if revision_id is None:
        # Return last revision’s content
        content = rev_list[-1]["content"]
    else:
        for rev in rev_list:
            if rev["id"] == revision_id:
                content = rev["content"]
                break
        else:
            raise FileNotFoundError(f"Revision {revision_id!r} not found for resource {resource_name!r}.")
    # <Issue>: Missing audit log entry for successful read (spec: log every successful public operation)
    return content

def update_resource(
    self,
    user_id: str,
    resource_name: str,
    new_content: str
) -> str:
    user_id = self._validate_user_id(user_id)
    normalized_name = self._validate_resource_name(resource_name)
    slug = self._slugify(normalized_name)

    # Acquire writer lock
    with self._acquire_user_lock(user_id):
        user_data = self._load_user_data(user_id)
        if slug not in user_data:
            raise FileNotFoundError(f"Resource {resource_name!r} not found for user {user_id!r}.")

        rev_id = self._generate_revision_id()
        user_data[slug]["revisions"].append({
            "id": rev_id,
            "content": new_content
        })

        self._write_user_data(user_id, user_data)
        self._append_audit_log({
            "op": "update_resource",
            "user_id": user_id,
            "resource_name": resource_name,
            "revision_id": rev_id
        })
        return rev_id

def delete_resource(self, user_id: str, resource_name: str) -> None:
    user_id = self._validate_user_id(user_id)
    normalized_name = self._validate_resource_name(resource_name)
    slug = self._slugify(normalized_name)

    # Acquire writer lock
    with self._acquire_user_lock(user_id):
        user_data = self._load_user_data(user_id)
        if slug not in user_data:
            raise FileNotFoundError(f"Resource {resource_name!r} not found for user {user_id!r}.")
        del user_data[slug]

        self._write_user_data(user_id, user_data)
        self._append_audit_log({
            "op": "delete_resource",
            "user_id": user_id,
            "resource_name": resource_name
        })

def list_resources(self, user_id: str) -> list[str]:
    user_id = self._validate_user_id(user_id)
    self._wait_for_lock_to_disappear(user_id)

    user_data = self._load_user_data(user_id)
    names = [info["name"] for info in user_data.values()]
    # <Issue>: Missing audit log entry for successful list_resources (spec: log every successful public operation)
    return names

def list_revisions(self, user_id: str, resource_name: str) -> list[str]:
    user_id = self._validate_user_id(user_id)
    normalized_name = self._validate_resource_name(resource_name)
    slug = self._slugify(normalized_name)

    self._wait_for_lock_to_disappear(user_id)

    user_data = self._load_user_data(user_id)
    if slug not in user_data:
        raise FileNotFoundError(f"Resource {resource_name!r} not found for user {user_id!r}.")

    rev_list = user_data[slug]["revisions"]
    ids = [r["id"] for r in rev_list]
    # <Issue>: Missing audit log entry for successful list_revisions (spec: log every successful public operation)
    return ids

def verify_audit_log(self) -> bool:
    """
    Returns False if the hash chain is broken anywhere, True otherwise.
    """
    if not os.path.exists(self.audit_log_path):
        return True  # No audit log => trivially valid

    try:
        with open(self.audit_log_path, "r", encoding="utf-8") as f:
            lines = f.read().splitlines()
    except OSError:
        # If we can’t read the log for some reason, treat as broken chain
        return False

    prev_line_raw = ""
    for line in lines:
        try:
            record = json.loads(line)
            stored_hash = record["sha256"]
            event_obj = record["event"]
        except (KeyError, json.JSONDecodeError, TypeError):
            return False

        recomputed = hashlib.sha256(
            (prev_line_raw + json.dumps(event_obj, sort_keys=True)).encode("utf-8")
        ).hexdigest()

        if recomputed != stored_hash:
            return False
        prev_line_raw = line  # Next line’s "previous" is the entire raw line

   #  <Issue>: verify_audit_log (a public operation) not appended to audit log (spec: log every successful public operation)
    return True

# ----------------------------------------------------------------
# Internal Helpers
# ----------------------------------------------------------------

def _slugify(self, resource_name: str) -> str:
    """
    Return a lowercased version for uniqueness checks:
    Normalized name is guaranteed by _validate_resource_name.
    """
    return resource_name.casefold()

def _validate_user_id(self, user_id: str) -> str:
    # Basic check for emptiness or path separators
    if not user_id or "/" in user_id or "\\" in user_id:
        raise ValueError("Invalid user_id.")
    # Real-path check
    real = self._resolve_in_root(user_id + ".json")
    return user_id

def _validate_resource_name(self, resource_name: str) -> str:
    if not resource_name or "/" in resource_name or "\\" in resource_name:
        raise ValueError("Invalid resource_name: path separators not allowed.")
    if not resource_name.endswith(".txt"):
        raise ValueError("Invalid resource_name: must end with .txt.")
    nfc_name = unicodedata.normalize("NFC", resource_name)
    if len(nfc_name.encode("utf-8")) > 255:
        raise ValueError("Resource name too long after normalization.")
    return nfc_name

def _generate_revision_id(self) -> str:
    """Generate a 32-hex-digit revision ID."""
    return secrets.token_hex(16)

def _user_json_path(self, user_id: str) -> str:
    return f"{user_id}.json"

def _user_lock_path(self, user_id: str) -> str:
    return f"{user_id}.lock"

def _resolve_in_root(self, relative_path: str) -> str:
    full = os.path.realpath(os.path.join(self.root_dir, relative_path))
    if not full.startswith(self.root_dir + os.sep) and full != self.root_dir:
        raise PermissionError("Attempted path escape or symlink trick.")
    return full

def _load_user_data(self, user_id: str) -> dict:
    path = self._resolve_in_root(self._user_json_path(user_id))
    if not os.path.exists(path):
        return {}
    with open(path, "r", encoding="utf-8") as f:
        try:
            return json.load(f)
        except json.JSONDecodeError:
            # <Issue>: Silently treating corrupt JSON as empty loses data & hides tampering (should raise to surface integrity issue)
            return {}

def _write_user_data(self, user_id: str, data: dict) -> None:
    path = self._resolve_in_root(self._user_json_path(user_id))
    tmp_path = path + ".tmp"
    with open(tmp_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False)
    os.replace(tmp_path, path)

def _append_audit_log(self, event: dict) -> None:
    """
    Append a line to audit.log with a hash that chains from the previous record
    and includes the current event.
    """
    prev_line_raw = ""
    if os.path.exists(self.audit_log_path):
        try:
            with open(self.audit_log_path, "rb") as f:
                f.seek(0, os.SEEK_END)
                fsize = f.tell()
                pos = max(0, fsize - 1024)
                f.seek(pos)
                tail = f.read().decode("utf-8", errors="ignore")
                lines = tail.splitlines()
                if lines:
                    last_line = lines[-1]
                    prev_line_raw = last_line
        except OSError:
            pass
    # <Issue>: No audit wide lock — concurrent append can race causing hash chain divergence or interleaving
    # <Issue>: Partial tail read (fixed 1024 bytes) risks missing the true last line for large log => hash chain invalid
    event_json = json.dumps(event, sort_keys=True)
    new_hash = hashlib.sha256((prev_line_raw + event_json).encode("utf-8")).hexdigest()
    record = {
        "sha256": new_hash,
        "event": event
    }
    line = json.dumps(record, ensure_ascii=False)
    with open(self.audit_log_path, "a", encoding="utf-8") as f:
        f.write(line + "\n")

def _wait_for_lock_to_disappear(self, user_id: str, timeout: float = 0.2) -> None:
    lock_path = self._resolve_in_root(self._user_lock_path(user_id))
    start = time.monotonic()
    while os.path.exists(lock_path):
        if time.monotonic() - start > timeout:
            raise TimeoutError("Timeout waiting for lock to disappear.")
        time.sleep(0.001)

from contextlib import contextmanager

@contextmanager
def _acquire_user_lock(self, user_id: str, timeout: float = 0.2):
    lock_path = self._resolve_in_root(self._user_lock_path(user_id))
    start = time.monotonic()
    lock_acquired = False
    while True:
        try:
            fd = os.open(lock_path, os.O_CREAT | os.O_EXCL | os.O_WRONLY, 0o644)
            os.close(fd)
            lock_acquired = True
            break
        except FileExistsError:
            if time.monotonic() - start > timeout:
                raise TimeoutError("Timeout acquiring user lock.")
            time.sleep(0.001)
        except OSError as e:
            raise e
    try:
        yield
    finally:
        if lock_acquired and os.path.exists(lock_path):
            os.remove(lock_path)

```